In [1]:
import os
import sys
import random
import math
import numpy as np
import argparse
import itertools
import glob
import datetime
import time
from PIL import Image
import matplotlib.pyplot as plt
import functools

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable
from torch import Tensor
from torch.nn import Parameter

from torch.nn.utils.parametrizations import spectral_norm as SpectralNorm
from piqa import SSIM

from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

In [2]:
def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)
    rgb_image.paste(image)
    return rgb_image


class ImageDataset(Dataset):
    def __init__(self, root, transforms1=None, transforms2=None, unaligned=False, mode="train"):
        self.transform1 = transforms.Compose(transforms1)
        self.transform2 = transforms.Compose(transforms2)
        self.unaligned = unaligned
    
        path_A = f'{root}/{mode}/A'
        path_B = f'{root}/{mode}/B'
        
        files_A = os.listdir(path_A)
        files_B = os.listdir(path_B)
        
        self.files_A = []        
        for file in files_A:
            # make sure file is an image
            if file.endswith(('.jpg', '.png', 'jpeg')):
                self.files_A.append(path_A+'/'+file)        
                
        self.files_B = []        
        for file in files_B:
            # make sure file is an image
            if file.endswith(('.jpg', '.png', 'jpeg')):
                self.files_B.append(path_B+'/'+file)
        
    def __getitem__(self, index):
        
        image_A = Image.open(self.files_A[index % len(self.files_A)])

        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert grayscale images to rgb
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)

        item_A = self.transform1(image_A)
        item_B = self.transform2(image_B)
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

In [3]:
class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))

In [4]:
class UnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
        X -------------------identity----------------------
        |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet submodule with skip connections.

        Parameters:
            outer_nc (int) -- the number of filters in the outer conv layer
            inner_nc (int) -- the number of filters in the inner conv layer
            input_nc (int) -- the number of channels in input images/features
            submodule (UnetSkipConnectionBlock) -- previously defined submodules
            outermost (bool)    -- if this module is the outermost module
            innermost (bool)    -- if this module is the innermost module
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
        """
        super(UnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        if input_nc is None:
            input_nc = outer_nc
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=use_bias)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = down + [submodule] + up
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = down + up
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=use_bias)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = down + [submodule] + up + [nn.Dropout(0.5)]
            else:
                model = down + [submodule] + up

        self.model = nn.Sequential(*model)

    def forward(self, x):
        if self.outermost:
            return self.model(x)
        else:   # add skip connections
            return torch.cat([x, self.model(x)], 1)

In [5]:
class UnetGenerator(nn.Module):
    """Create a Unet-based generator"""

    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        """Construct a Unet generator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            output_nc (int) -- the number of channels in output images
            num_downs (int) -- the number of downsamplings in UNet. For example, # if |num_downs| == 7,
                                image of size 128x128 will become of size 1x1 # at the bottleneck
            ngf (int)       -- the number of filters in the last conv layer
            norm_layer      -- normalization layer

        We construct the U-Net from the innermost layer to the outermost layer.
        It is a recursive process.
        """
        super(UnetGenerator, self).__init__()
        # construct unet structure
        unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True)  # add the innermost layer
        for i in range(num_downs - 5):          # add intermediate layers with ngf * 8 filters
            unet_block = UnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout)
        # gradually reduce the number of filters from ngf * 8 to ngf
        unet_block = UnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        unet_block = UnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer)
        self.model = UnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer)  # add the outermost layer

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [6]:
class DiscriminatorSN(nn.Module):
    def __init__(self, input_nc, ndf=64, n_layers=3, use_sigmoid=False):
        super(DiscriminatorSN, self).__init__()
        use_bias = False

        kw = 4
        padw = 1
        sequence = [
            SpectralNorm(nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw)),
            nn.LeakyReLU(0.2, True)
        ]

        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):
            nf_mult_prev = nf_mult
            nf_mult = min(2**n, 8)
            sequence += [
                SpectralNorm(nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                          kernel_size=kw, stride=2, padding=padw, bias=use_bias)),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2**n_layers, 8)
        sequence += [
            SpectralNorm(nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult,
                      kernel_size=kw, stride=1, padding=padw, bias=use_bias)),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [SpectralNorm(nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw))]

        if use_sigmoid:
            sequence += [nn.Sigmoid()]

        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        return self.model(input)

In [7]:
def f(output_size, ksize, stride):
    return (output_size - 1) * stride + ksize

last_layer = f(output_size=1, ksize=4, stride=1)
# Receptive field: 4
fourth_layer = f(output_size=last_layer, ksize=4, stride=1)
# Receptive field: 7
third_layer = f(output_size=fourth_layer, ksize=4, stride=2)
# Receptive field: 16
second_layer = f(output_size=third_layer, ksize=4, stride=2)
# Receptive field: 34
first_layer = f(output_size=second_layer, ksize=4, stride=2)
# Receptive field: 70

print(first_layer)

70


In [8]:
epoch = 0
n_epochs = 125
dataset_name = "cartoon2pixelart"
experiment = "unet_sn_perc"
batch_size = 1
lr = 0.0002 # Adam learning rate
b1 = 0.5 # beta1. Adam decay of first order momentum of gradient
b2 = 0.999 # beta2. Adam decay of first order momentum of gradient
decay_epoch = 50 # from which epoch to begin lr decay
n_cpu = 8 # number of CPU threads. aka dataloader workers
image_height = 256
image_width = 256
channels = 3
sample_interval = 1000
checkpoint_interval = 30
n_residual_blocks = 9
lambda_cyc = 10.0
lambda_id =10.0
discriminator_output_shape = (1,30,30)


# Create sample and checkpoint directories
os.makedirs("images/%s/%s" % (dataset_name, experiment), exist_ok=True)
os.makedirs("saved_models/%s/%s" % (dataset_name, experiment), exist_ok=True)

# Losses
criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

cuda = torch.cuda.is_available()

input_shape = (channels, image_height, image_width)

# Initialize generator and discriminator
G_AB = UnetGenerator(3,3,8)
G_BA = UnetGenerator(3,3,8)
D_A = DiscriminatorSN(channels)
D_B = DiscriminatorSN(channels)



if cuda:
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()
    
print(G_AB)
print(D_A)
    
def weights_init_normal(m):
    """
    Initialize convolution layer weights to $\mathcal{N}(0, 0.2)$
    """
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)

if epoch != 0:
    # Load pretrained models
    G_AB.load_state_dict(torch.load("saved_models/%s/%s/G_AB_%d.pth" % (dataset_name, experiment, epoch)))
    G_BA.load_state_dict(torch.load("saved_models/%s/%s/G_BA_%d.pth" % (dataset_name, experiment, epoch)))
    D_A.load_state_dict(torch.load("saved_models/%s/%s/D_A_%d.pth" % (dataset_name, experiment, epoch)))
    D_B.load_state_dict(torch.load("saved_models/%s/%s/D_B_%d.pth" % (dataset_name, experiment, epoch)))
else:
    # Initialize weights
    G_AB.apply(weights_init_normal)
    G_BA.apply(weights_init_normal)
    D_A.apply(weights_init_normal)
    D_B.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))


# Learning rate update schedulers
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=lambda e: 1.0 - max(0, e - decay_epoch) / (n_epochs - decay_epoch)
)

lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=lambda e: 1.0 - max(0, e - decay_epoch) / (n_epochs - decay_epoch)
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=lambda e: 1.0 - max(0, e - decay_epoch) / (n_epochs - decay_epoch)
)

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

# Image transformations
transforms1 = [
    transforms.Resize(int(image_height), transforms.InterpolationMode.BICUBIC),
  #  transforms.RandomCrop((192, 192)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

transforms2 = [
    transforms.Resize(int(image_height), transforms.InterpolationMode.NEAREST),
 #   transforms.RandomCrop((192, 192)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]


# Training data loader
dataloader = DataLoader(
    ImageDataset("dataset/%s" % dataset_name, transforms1=transforms1,transforms2=transforms2, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
)
# Test data loader
val_dataloader = DataLoader(
    ImageDataset("dataset/%s" % dataset_name, transforms1=transforms1,transforms2=transforms2, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)



def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))
    G_AB.eval()
    #G_BA.eval()
    real_A = Variable(imgs["A"].type(Tensor))
    fake_B = G_AB(real_A)
    fake_B.cpu()
    real_B = Variable(imgs["B"].type(Tensor))
    real_B.cpu()
    real_A.cpu()
    #fake_A = G_BA(real_B)
    #fake_A.cpu()
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)
    real_B = make_grid(real_B, nrow=5, normalize=True)
    #fake_A = make_grid(fake_A, nrow=5, normalize=True)
    fake_B = make_grid(fake_B, nrow=5, normalize=True)
    # Arange images along y-axis
    #image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)
    image_grid = torch.cat((real_A, fake_B, real_B), 1)
    save_image(image_grid, "images/%s/%s/%s.png" % (dataset_name, experiment ,batches_done), normalize=False)

from torchvision import models


class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['9', '36']
        self.vgg = models.vgg19(weights='DEFAULT').features

    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features[0], features[1]

vgg = VGGNet().cuda().eval()

# ----------
#  Training
# ----------
print(torch.cuda.current_device())

print(torch.cuda.device_count())
# setting device on GPU if available, else CPU


#Additional Info when using cuda
if cuda:
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
print(os.path.exists("dataset/cartoon2pixelart/train/A"))

epoch_losses_G_AB = []
epoch_losses_G_BA = []
epoch_losses_D_A = []
epoch_losses_D_B = []
epoch_losses_D = []
epoch_losses_G = []
epoch_losses_GAN = []
epoch_losses_cycle = []
epoch_losses_identity = []


torch.cuda.empty_cache()
prev_time = time.time()
for epoch in range(epoch, n_epochs):
    losses_G_AB = []
    losses_G_BA = []
    losses_D_A = []
    losses_D_B = []
    losses_D = []
    losses_G = []
    losses_GAN = []
    losses_cycle = []
    losses_identity = []

   
    for i, batch in enumerate(dataloader):
        

        # Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))

        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *discriminator_output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *discriminator_output_shape))), requires_grad=False)

        # ------------------
        #  Train Generators
        # ------------------

        G_AB.train()
        G_BA.train()

        optimizer_G.zero_grad()

        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)

        loss_identity = (loss_id_A + loss_id_B) / 2

        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2

        # Cycle loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2
        

        
        rx = recov_A
        ry = recov_B

        fx1, fx2 = vgg(real_A)
        
        fy1, fy2 = vgg(real_B)

        frx1, frx2 = vgg(rx)
        fry1, fry2 = vgg(ry)
        
        m1 = criterion_GAN(fx1, frx1)
        m2 = criterion_GAN(fx2, frx2)

        m3 = criterion_GAN(fy1, fry1)
        m4 = criterion_GAN(fy2, fry2)

        loss_perceptual = (m1 + m2 + m3 + m4) * 0.01



        # Total loss
        loss_G = loss_GAN + lambda_cyc * (loss_cycle+loss_perceptual) + lambda_id * loss_identity

        loss_G.backward()
        optimizer_G.step()

        # -----------------------
        #  Train Discriminator A
        # -----------------------

        optimizer_D_A.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
        # Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
        # Total loss
        loss_D_A = (loss_real + loss_fake) / 2

        loss_D_A.backward()
        optimizer_D_A.step()

        # -----------------------
        #  Train Discriminator B
        # -----------------------

        optimizer_D_B.zero_grad()

        # Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
        # Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
        # Total loss
        loss_D_B = (loss_real + loss_fake) / 2

        loss_D_B.backward()
        optimizer_D_B.step()

        loss_D = (loss_D_A + loss_D_B) / 2

        # --------------
        #  Log Progress
        # --------------

        # Determine approximate time left
        batches_done = epoch * len(dataloader) + i
        batches_left = n_epochs * len(dataloader) - batches_done
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
        prev_time = time.time()
        
        losses_G_AB.append(loss_GAN_AB.item())
        losses_G_BA.append(loss_GAN_BA.item())
        losses_D_A.append(loss_D_A.item())
        losses_D_B.append(loss_D_B.item())
        losses_D.append(loss_D.item())
        losses_G.append(loss_G.item())
        losses_GAN.append(loss_GAN.item())
        losses_cycle.append(loss_cycle.item())
        losses_identity.append(loss_identity.item())
        

        # Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f, perceptual: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                loss_perceptual.item(),
                time_left,
            )
        )

        # If at sample interval save image
        if batches_done % sample_interval == 0:
            sample_images(batches_done)
            
    epoch_losses_G_AB.append(sum(losses_G_AB)/len(losses_G_AB))
    epoch_losses_G_BA.append(sum(losses_G_BA)/len(losses_G_BA))
    epoch_losses_D_A.append(sum(losses_D_A)/len(losses_D_A))
    epoch_losses_D_B.append(sum(losses_D_B)/len(losses_D_B))
    
    epoch_losses_D.append(sum(losses_D)/len(losses_D))
    epoch_losses_G.append(sum(losses_G)/len(losses_G))
    epoch_losses_GAN.append(sum(losses_GAN)/len(losses_GAN))
    epoch_losses_cycle.append(sum(losses_cycle)/len(losses_cycle))
    epoch_losses_identity.append(sum(losses_identity)/len(losses_identity))
 
 

    # Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()

    if checkpoint_interval != -1 and epoch % checkpoint_interval == 0:
        # Save model checkpoints
        torch.save(G_AB.state_dict(), "saved_models/%s/%s/G_AB_%d.pth" % (dataset_name, experiment, epoch))
        torch.save(G_BA.state_dict(), "saved_models/%s/%s/G_BA_%d.pth" % (dataset_name, experiment, epoch))
        torch.save(D_A.state_dict(), "saved_models/%s/%s/D_A_%d.pth" % (dataset_name, experiment, epoch))
        torch.save(D_B.state_dict(), "saved_models/%s/%s/D_B_%d.pth" % (dataset_name, experiment, epoch))
        #losses = [epoch_losses_D, epoch_losses_G, epoch_losses_GAN, epoch_losses_cycle, epoch_losses_identity, epoch_losses_perceptual, epoch_losses_G_AB, epoch_losses_G_BA, epoch_losses_D_A, epoch_losses_D_B]
        

        #with open(f'cyclegan_perceptual_losses_id_{epoch}', 'wb') as fp:
         #   pickle.dump(losses, fp)

UnetGenerator(
  (model): UnetSkipConnectionBlock(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): UnetSkipConnectionBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2, inplace=True)
          (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
          (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): UnetSkipConnectionBlock(
            (model): Sequential(
              (0): LeakyReLU(negative_slope=0.2, inplace=True)
              (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
              (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (3): UnetSkipConnectionBlock(
                (model): Sequential(
                  (0): LeakyReLU(negative_slope=0.2, inplace=True)
                  (1): Conv2d(256, 512,

In [9]:
losses = [epoch_losses_D, epoch_losses_G, epoch_losses_GAN, epoch_losses_cycle, epoch_losses_identity, epoch_losses_ssim, epoch_losses_G_AB, epoch_losses_G_BA, epoch_losses_D_A, epoch_losses_D_B]

NameError: name 'epoch_losses_ssim' is not defined

In [ ]:
import pickle

with open('cyclegan_unet_II', 'wb') as fp:
    pickle.dump(losses, fp)
    print('Done writing list into a binary file')